# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium

In [1]:
!pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Abri Navegador e entrar no Google
navegador = webdriver.Chrome()
td_cotas = ['Cotação Dolar', 'Cotação Euro']
# Cotação Dolar
navegador.get('https://www.google.com/')
pesquisa = navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
pesquisa.send_keys('Cotação Dolar')
pesquisa.send_keys(Keys.ENTER)
cota_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]')
cota_dolar = cota_dolar.get_attribute('data-value') # Ler atributo de valor de cotação
print(cota_dolar)
# Cotação Euro
navegador.get('https://www.google.com/')
pesquisa = navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
pesquisa.send_keys('Cotação Euro')
pesquisa.send_keys(Keys.ENTER)
cota_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]')
cota_euro = cota_euro.get_attribute('data-value') # Ler atributo de valor de cotação
print(cota_euro)
# Cotação Ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cota_ouro = navegador.find_element('xpath', '//*[@id="comercial"]')
cota_ouro = cota_ouro.get_attribute('value') # Ler atributo de valor de cotação
cota_ouro = cota_ouro.replace(',', '.') # Substitui a virgula por ponto
print(cota_ouro)

navegador.quit()

5.366099999999999
5.5667384790000005
302.48


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [4]:
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [5]:
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cota_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cota_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cota_ouro)
# Calculando os valores das colunas
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.366100,5366.046339,1.40,7512.464875
1,Carro Renault,4500.00,Euro,5.566738,25050.323156,2.00,50100.646311
2,Notebook Dell,899.99,Dólar,5.366100,4829.436339,1.70,8210.041776
3,IPhone,799.00,Dólar,5.366100,4287.513900,1.70,7288.773630
4,Carro Fiat,3000.00,Euro,5.566738,16700.215437,1.90,31730.409330
5,Celular Xiaomi,480.48,Dólar,5.366100,2578.303728,2.00,5156.607456
6,Joia 20g,20.00,Ouro,302.480000,6049.600000,1.15,6957.040000


### Agora vamos exportar a nova base de preços atualizada

In [7]:
tabela.to_excel('Produtos Novo.xlsx', index=False)